In [1]:
import datetime
import numpy as np
import pandas as pd
import vectorbt as vbt
import plotly.graph_objects as go

In [18]:
end_time = '2022-09-30'

start_time = '2021-09-30'

In [19]:
RRRP3 = vbt.YFData.download(
    "RRRP3.SA",
    missing_index="drop",
    interval = "1d",
    start = start_time,
    end = end_time
)

RRRP3_price = RRRP3.get("Close")

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning:

The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html



In [20]:
def plot_bbands(vbt_stock):
    """
    Plots the BBANDS of a vbt object.
    """
    df = vbt_stock.get().reset_index()

    for i in ['Open', 'High', 'Close', 'Low']:
        df[i]  =  df[i].astype('float64')

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                         open=df['Open'],
                                         high=df['High'],
                                         low=df['Low'],
                                        close=df['Close'])])

    return fig.show()

In [21]:
plot_bbands(RRRP3)

In [22]:
# but also using the native plot_bbands function instead

bbands = vbt.BBANDS.run(RRRP3_price)

# using jupyter
bbands.plot()

FigureWidget({
    'data': [{'line': {'color': 'rgba(127,127,127,0.7500)'},
              'name': 'Lower Band'…

In [23]:
# rsi strategy
rsi = vbt.RSI.run(RRRP3_price, window = 21)

entries = rsi.rsi_crossed_below(30)

exits = rsi.rsi_crossed_above(70)

pf = vbt.Portfolio.from_signals(RRRP3_price, entries, exits)

print(pf.stats())

pf.plot().show()

Start                         2021-09-30 03:00:00+00:00
End                           2022-09-29 03:00:00+00:00
Period                                              250
Start Value                                       100.0
End Value                                    143.341516
Total Return [%]                              43.341516
Benchmark Return [%]                         -17.699524
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              21.385378
Max Drawdown Duration                             107.0
Total Trades                                          2
Total Closed Trades                                   2
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                      100.0
Best Trade [%]                                32.246375
Worst Trade [%]                                8

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [30]:
fast_ma = vbt.MA.run(RRRP3_price, window = 21)
slow_ma = vbt.MA.run(RRRP3_price, window = 210)

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(RRRP3_price, entries, exits)

print(pf.stats())
pf.plot().show()

Start                         2021-09-30 03:00:00+00:00
End                           2022-09-29 03:00:00+00:00
Period                                              250
Start Value                                       100.0
End Value                                     92.021005
Total Return [%]                              -7.978995
Benchmark Return [%]                         -17.699524
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               12.08903
Max Drawdown Duration                               5.0
Total Trades                                          1
Total Closed Trades                                   0
Total Open Trades                                     1
Open Trade PnL                                -7.978995
Win Rate [%]                                        NaN
Best Trade [%]                                      NaN
Worst Trade [%]                                 

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\joaop\Anaconda3\envs\vbt\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [41]:
windows = np.arange(2, 101)
fast_ma, slow_ma = vbt.MA.run_combs(RRRP3_price, window=windows, r=2, short_names=['fast', 'slow'])
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')
pf = vbt.Portfolio.from_signals(RRRP3_price, entries, exits, **pf_kwargs)

In [54]:
returns = pf.total_return()

print(returns.max())
print(returns.idxmax())

0.25208858230808334
(9, 10)


In [56]:
pf[(9, 10)].stats()

Start                         2021-09-30 03:00:00+00:00
End                           2022-09-29 03:00:00+00:00
Period                                250 days 00:00:00
Start Value                                       100.0
End Value                                    125.208858
Total Return [%]                              25.208858
Benchmark Return [%]                         -17.699524
Max Gross Exposure [%]                            100.0
Total Fees Paid                                5.203232
Max Drawdown [%]                              12.874798
Max Drawdown Duration                  69 days 00:00:00
Total Trades                                         22
Total Closed Trades                                  22
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  59.090909
Best Trade [%]                                10.522086
Worst Trade [%]                              -11

In [57]:
pf[(9, 10)].plot().show()